<a href="https://colab.research.google.com/github/shibakyrc0123-arch/A-Gps-Versiones/blob/main/Ver_1_1_A_Gps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
from datetime import timedelta
import time

# --- 1. BUSCADOR AUTOMÁTICO ---
def encontrar_archivo():
    rutas_a_buscar = ['/content/', '/content/sample_data/']
    print("🔍 Buscando archivo...")
    for ruta_base in rutas_a_buscar:
        if os.path.exists(ruta_base):
            archivos = os.listdir(ruta_base)
            for archivo in archivos:
                if (archivo.endswith('.xls') or archivo.endswith('.xlsx') or archivo.endswith('.csv')) \
                   and "california_housing" not in archivo and "mnist" not in archivo:
                    return os.path.join(ruta_base, archivo)
    return None

# --- CONFIGURACIÓN ---
bodega_coords = [
    (4.101740989185771, -73.65614987790829),
    (4.10275779142336, -73.65779357282884),
    (4.103576674506101, -73.65722519691684),
    (4.103303969059117, -73.65525092470286)
]
bodega_polygon = Polygon(bodega_coords)
geolocator = Nominatim(user_agent="tracker_final_v3")

def obtener_direccion(lat, lon):
    try:
        time.sleep(1.1)
        location = geolocator.reverse((lat, lon), exactly_one=True)
        return location.address if location else "Ubicación desconocida"
    except:
        return "Error geoloc"

def procesar_logica_vehiculo(archivo):
    print(f"📂 Procesando: {archivo}")

    # 1. LECTURA
    try:
        if archivo.endswith('.csv'):
            df = pd.read_csv(archivo)
        else:
            df = pd.read_excel(archivo)
    except Exception as e:
        return f"🔥 Error leyendo archivo: {e}"

    # --- CORRECCIÓN DE MAYÚSCULAS/MINÚSCULAS ---
    # Convertimos todos los nombres de columnas a MAYÚSCULAS y quitamos espacios
    df.columns = df.columns.str.upper().str.strip()

    print(f"✅ Columnas detectadas: {list(df.columns)}")

    # Validamos usando los nombres en MAYÚSCULAS
    if 'EVENTO' not in df.columns or 'COORDS' not in df.columns:
        return f"❌ Faltan columnas. Se requiere EVENTO y COORDS."

    # Aseguramos formato fecha (FHEVENTO)
    df['FHEVENTO'] = pd.to_datetime(df['FHEVENTO'])